In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pandas.io import sql
import snowflake.connector
import keyring
import psycopg2 
import time
from datetime import date, timedelta
from scipy import stats

pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', 500)

from matplotlib import pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
sns.set_style("darkgrid")

In [2]:
snowflake_username = 'matthew.bessey@disneystreaming.com'

In [3]:
ctx = snowflake.connector.connect(authenticator='externalbrowser', 
                                  user=snowflake_username, 
                                  account='disneystreaming.us-east-1')

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [6]:
# set date parameters for query

subscription_start_date_min = "'2019-12-01'"
subscription_start_date_max = "'2019-12-08'" # max of subscription start date
engagement_date = "'2019-12-14'" # date for which we want to pull engagement behaviors

In [48]:
query= """
select o.swid
, o.swid_holdout
, a.accountid
, s.subscription_id
, e.*
from subscription s
join account a on s.account_id = a.accountid
join oneid_combined o on a.swid = o.swid
join "DSS_PROD"."DISNEY_PLUS"."DIM_DISNEY_DAILY_ACCOUNT_ENGAGEMENT" e on a.accountid = e.account_id
where s.partner = 'disney'
and s.calculated_subscription_start_dtm >= {}
and s.CALCULATED_SUBSCRIPTION_START_DTM <= {}
--and s.is_entitled = 1
and e.ds = {}
and e.is_pre_launch != 1
limit 500000;
""".format(subscription_start_date_min, subscription_start_date_max, engagement_date)

In [49]:
# run the query and write to engagement
engagement = pd.read_sql(query,ctx)

In [50]:
# rename columns and preview data
engagement.columns = engagement.columns.str.lower()
engagement.head()

,swid,swid_holdout,accountid,subscription_id,ds,account_id,is_flagged,first_account_subscription_signup_date,first_account_subscription_signup_week,last_account_subscription_signup_date,last_account_subscription_signup_week,is_entitled,is_entitled_l1,is_entitled_l7,is_entitled_l28,is_entitled_itd,is_pre_launch,is_pre_launch_nltt,account_home_country,subscription_state_upd,subscription_type,total_login_days_l1,total_login_days_l7,total_login_days_l28,total_login_days_itd,total_streams_l1,total_streams_l7,total_streams_l28,total_streams_itd,num_streaming_profiles_l1,num_streaming_profiles_l7,num_streaming_profiles_l28,num_streaming_profiles_itd,num_general_streaming_profiles_l1,num_general_streaming_profiles_l7,num_general_streaming_profiles_l28,num_general_streaming_profiles_itd,num_kids_streaming_profiles_l1,num_kids_streaming_profiles_l7,num_kids_streaming_profiles_l28,num_kids_streaming_profiles_itd,num_streaming_devices_l1,num_streaming_devices_l7,num_streaming_devices_l28,num_streaming_devices_itd,account_total_stream_days_l1,account_total_stream_days_l7,account_total_stream_days_l28,account_total_stream_days_itd,account_profile_total_stream_days_l1,account_profile_total_stream_days_l7,account_profile_total_stream_days_l28,account_profile_total_stream_days_itd,total_stream_days_general_profiles_l1,total_stream_days_general_profiles_l7,total_stream_days_general_profiles_l28,total_stream_days_general_profiles_itd,total_stream_days_kids_profiles_l1,total_stream_days_kids_profiles_l7,total_stream_days_kids_profiles_l28,total_stream_days_kids_profiles_itd,total_stream_time_ms_l1,total_stream_time_ms_l7,total_stream_time_ms_l28,total_stream_time_ms_itd,total_stream_time_general_profiles_ms_l1,total_stream_time_general_profiles_ms_l7,total_stream_time_general_profiles_ms_l28,total_stream_time_general_profiles_ms_itd,total_stream_time_kids_profiles_ms_l1,total_stream_time_kids_profiles_ms_l7,total_stream_time_kids_profiles_ms_l28,total_stream_time_kids_profiles_ms_itd,total_stream_time_web_ms_l1,total_stream_time_web_ms_l7,total_stream_time_web_ms_l28,total_stream_time_web_ms_itd,total_stream_time_mobile_ms_l1,total_stream_time_mobile_ms_l7,total_stream_time_mobile_ms_l28,total_stream_time_mobile_ms_itd,total_stream_time_connected_tv_ms_l1,total_stream_time_connected_tv_ms_l7,total_stream_time_connected_tv_ms_l28,total_stream_time_connected_tv_ms_itd,total_stream_time_unknown_ms_l1,total_stream_time_unknown_ms_l7,total_stream_time_unknown_ms_l28,total_stream_time_unknown_ms_itd,last_stream_date,days_since_last_stream
0,{A1881602-4957-49C4-9706-DF6AC44951CE},15,75a57224-e9ba-4463-a495-05aadcf98579,D2C:BAMTECH:urn:dss:disney:orders:1e9260b5-5d7...,2019-12-14,75a57224-e9ba-4463-a495-05aadcf98579,0,2019-12-02,2019-12-02,2019-12-02,2019-12-02,1,1,1,1,1,0,0,US,All,All,1,2,7,7,1,7,15,15,1,1,1,1,1,1,1,1,0,0,0,0,1,2,2,2,1,2,7,7,1,2,7,7,1,2,7,7,0,0,0,0,2790503,19180218,39967530,39967530,2790503,19180218,39967530,39967530,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-14,0.0
1,{A19AA3FF-DC27-4552-A336-46C30247842C},223,d55c4240-5256-469c-adc5-adbd555bab5d,D2C:BAMTECH:urn:dss:disney:orders:db587ad2-e5a...,2019-12-14,d55c4240-5256-469c-adc5-adbd555bab5d,0,2019-12-02,2019-12-02,2019-12-02,2019-12-02,1,1,1,1,1,0,0,US,All,All,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,NaN
2,{A18078AA-EE6E-4508-8FB2-7AEA9F4AC5C8},113,ae1b375b-c8fc-4426-bed6-199b776c77f5,EXT:VERIZON:VR-BNQRGH9QSKTNBTVZ,2019-12-14,ae1b375b-c8fc-4426-bed6-199b776c77f5,0,2019-12-03,2019-12-02,2019-12-03,2019-12-02,1,1,1,1,1,0,0,US,All,All,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,NaN
3,{A1A2CE86-8C90-4EB2-8B75-B1FE7DCF28C9},34,5e371f54-c47f-4919-90a2-eb53b86d661d,IAP:ROKU:3bb52c46-1ec0-4776-ae0a-ab1601855e08,2019-12-14,5e371f54-c47f-4919-90a2-eb53b86d661d,0,2019-12-01,2019-11-25,2019-12-01,

In [51]:
# create function and apply for mapping of holdout groups on 'swid_holdout'
def holdout_grouping(df):
    if df['swid_holdout'] < 243:
        return "all marketing"
    elif df['swid_holdout'] >= 243 and df['swid_holdout'] < 246:
        return "no onboarding"
    else:
        return "no marketing"
    
engagement['marketing_holdout'] = engagement.apply(holdout_grouping,axis=1)

In [52]:
# preview data for function execution
engagement.head()

,swid,swid_holdout,accountid,subscription_id,ds,account_id,is_flagged,first_account_subscription_signup_date,first_account_subscription_signup_week,last_account_subscription_signup_date,last_account_subscription_signup_week,is_entitled,is_entitled_l1,is_entitled_l7,is_entitled_l28,is_entitled_itd,is_pre_launch,is_pre_launch_nltt,account_home_country,subscription_state_upd,subscription_type,total_login_days_l1,total_login_days_l7,total_login_days_l28,total_login_days_itd,total_streams_l1,total_streams_l7,total_streams_l28,total_streams_itd,num_streaming_profiles_l1,num_streaming_profiles_l7,num_streaming_profiles_l28,num_streaming_profiles_itd,num_general_streaming_profiles_l1,num_general_streaming_profiles_l7,num_general_streaming_profiles_l28,num_general_streaming_profiles_itd,num_kids_streaming_profiles_l1,num_kids_streaming_profiles_l7,num_kids_streaming_profiles_l28,num_kids_streaming_profiles_itd,num_streaming_devices_l1,num_streaming_devices_l7,num_streaming_devices_l28,num_streaming_devices_itd,account_total_stream_days_l1,account_total_stream_days_l7,account_total_stream_days_l28,account_total_stream_days_itd,account_profile_total_stream_days_l1,account_profile_total_stream_days_l7,account_profile_total_stream_days_l28,account_profile_total_stream_days_itd,total_stream_days_general_profiles_l1,total_stream_days_general_profiles_l7,total_stream_days_general_profiles_l28,total_stream_days_general_profiles_itd,total_stream_days_kids_profiles_l1,total_stream_days_kids_profiles_l7,total_stream_days_kids_profiles_l28,total_stream_days_kids_profiles_itd,total_stream_time_ms_l1,total_stream_time_ms_l7,total_stream_time_ms_l28,total_stream_time_ms_itd,total_stream_time_general_profiles_ms_l1,total_stream_time_general_profiles_ms_l7,total_stream_time_general_profiles_ms_l28,total_stream_time_general_profiles_ms_itd,total_stream_time_kids_profiles_ms_l1,total_stream_time_kids_profiles_ms_l7,total_stream_time_kids_profiles_ms_l28,total_stream_time_kids_profiles_ms_itd,total_stream_time_web_ms_l1,total_stream_time_web_ms_l7,total_stream_time_web_ms_l28,total_stream_time_web_ms_itd,total_stream_time_mobile_ms_l1,total_stream_time_mobile_ms_l7,total_stream_time_mobile_ms_l28,total_stream_time_mobile_ms_itd,total_stream_time_connected_tv_ms_l1,total_stream_time_connected_tv_ms_l7,total_stream_time_connected_tv_ms_l28,total_stream_time_connected_tv_ms_itd,total_stream_time_unknown_ms_l1,total_stream_time_unknown_ms_l7,total_stream_time_unknown_ms_l28,total_stream_time_unknown_ms_itd,last_stream_date,days_since_last_stream,marketing_holdout
0,{A1881602-4957-49C4-9706-DF6AC44951CE},15,75a57224-e9ba-4463-a495-05aadcf98579,D2C:BAMTECH:urn:dss:disney:orders:1e9260b5-5d7...,2019-12-14,75a57224-e9ba-4463-a495-05aadcf98579,0,2019-12-02,2019-12-02,2019-12-02,2019-12-02,1,1,1,1,1,0,0,US,All,All,1,2,7,7,1,7,15,15,1,1,1,1,1,1,1,1,0,0,0,0,1,2,2,2,1,2,7,7,1,2,7,7,1,2,7,7,0,0,0,0,2790503,19180218,39967530,39967530,2790503,19180218,39967530,39967530,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-12-14,0.0,all marketing
1,{A19AA3FF-DC27-4552-A336-46C30247842C},223,d55c4240-5256-469c-adc5-adbd555bab5d,D2C:BAMTECH:urn:dss:disney:orders:db587ad2-e5a...,2019-12-14,d55c4240-5256-469c-adc5-adbd555bab5d,0,2019-12-02,2019-12-02,2019-12-02,2019-12-02,1,1,1,1,1,0,0,US,All,All,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,NaN,all marketing
2,{A18078AA-EE6E-4508-8FB2-7AEA9F4AC5C8},113,ae1b375b-c8fc-4426-bed6-199b776c77f5,EXT:VERIZON:VR-BNQRGH9QSKTNBTVZ,2019-12-14,ae1b375b-c8fc-4426-bed6-199b776c77f5,0,2019-12-03,2019-12-02,2019-12-03,2019-12-02,1,1,1,1,1,0,0,US,All,All,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,None,NaN,all marketing
3,{A1A2CE86-8C90-4EB2-8B75-B1FE7DCF28C9},34,5e371f54-c47f-4919-90a2-eb53b86d661d,IAP:ROKU:3bb52c46-1ec0-4776-ae0a-ab1601855e08,2019-12-14,5e371f54-c47

In [53]:
#drop rows w/ no entitlement data, rewrite as int
engagement = engagement.drop(engagement[engagement['is_entitled']=='unknown'].index,axis=0)
engagement.is_entitled = engagement.is_entitled.astype(int)

In [54]:
# create active_binary_l1 with 1 = user was active in l1d
engagement['active_binary_l1'] = engagement.apply(lambda df:
                                                 1 if df['total_streams_l1'] > 0
                                                 else 0,
                                                 axis=1)

In [55]:
# create active_binary_l7 with 1 = user was active in l7d
engagement['active_binary_l7'] = engagement.apply(lambda df:
                                                 1 if df['total_streams_l7'] > 0
                                                 else 0,
                                                 axis=1)

In [56]:
engagement.groupby('marketing_holdout').mean()

,swid_holdout,is_flagged,is_entitled,is_pre_launch,is_pre_launch_nltt,total_login_days_l1,total_login_days_l7,total_login_days_l28,total_login_days_itd,total_streams_l1,total_streams_l7,total_streams_l28,total_streams_itd,num_streaming_profiles_l1,num_streaming_profiles_l7,num_streaming_profiles_l28,num_streaming_profiles_itd,num_general_streaming_profiles_l1,num_general_streaming_profiles_l7,num_general_streaming_profiles_l28,num_general_streaming_profiles_itd,num_kids_streaming_profiles_l1,num_kids_streaming_profiles_l7,num_kids_streaming_profiles_l28,num_kids_streaming_profiles_itd,num_streaming_devices_l1,num_streaming_devices_l7,num_streaming_devices_l28,num_streaming_devices_itd,account_total_stream_days_l1,account_total_stream_days_l7,account_total_stream_days_l28,account_total_stream_days_itd,account_profile_total_stream_days_l1,account_profile_total_stream_days_l7,account_profile_total_stream_days_l28,account_profile_total_stream_days_itd,total_stream_days_general_profiles_l1,total_stream_days_general_profiles_l7,total_stream_days_general_profiles_l28,total_stream_days_general_profiles_itd,total_stream_days_kids_profiles_l1,total_stream_days_kids_profiles_l7,total_stream_days_kids_profiles_l28,total_stream_days_kids_profiles_itd,total_stream_time_ms_l1,total_stream_time_ms_l7,total_stream_time_ms_l28,total_stream_time_ms_itd,total_stream_time_general_profiles_ms_l1,total_stream_time_general_profiles_ms_l7,total_stream_time_general_profiles_ms_l28,total_stream_time_general_profiles_ms_itd,total_stream_time_kids_profiles_ms_l1,total_stream_time_kids_profiles_ms_l7,total_stream_time_kids_profiles_ms_l28,total_stream_time_kids_profiles_ms_itd,total_stream_time_web_ms_l1,total_stream_time_web_ms_l7,total_stream_time_web_ms_l28,total_stream_time_web_ms_itd,total_stream_time_mobile_ms_l1,total_stream_time_mobile_ms_l7,total_stream_time_mobile_ms_l28,total_stream_time_mobile_ms_itd,total_stream_time_connected_tv_ms_l1,total_stream_time_connected_tv_ms_l7,total_stream_time_connected_tv_ms_l28,total_stream_time_connected_tv_ms_itd,total_stream_time_unknown_ms_l1,total_stream_time_unknown_ms_l7,total_stream_time_unknown_ms_l28,total_stream_time_unknown_ms_itd,days_since_last_stream,active_binary_l1,active_binary_l7
marketing_holdout,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
all marketing,120.877450,0.000401,0.889411,0.0,0.002757,0.630070,4.441082,9.545721,9.864544,2.314522,15.887367,35.872923,37.130661,0.510543,1.060266,1.327884,1.331573,0.492187,1.020802,1.278033,1.281484,0.018357,0.040331,0.052794,0.053252,0.590254,1.499494,2.137012,2.158099,0.432520,3.013640,6.635228,6.844133,0.510543,3.517123,7.781292,8.030277,0.492187,3.400810,7.543755,7.783051,0.018357,0.116312,0.237537,0.247226,4.171823e+06,2.806338e+07,6.467702e+07,6.704882e+07,4.040303e+06,2.729760e+07,6.308764e+07,6.539365e+07,131519.995332,765781.238474,1.589382e+06,1.655176e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.544900,0.432520,0.782511
no marketing,250.521195,0.000359,0.889948,0.0,0.002717,0.630735,4.436516,9.461736,9.773130,2.342304,15.777129,35.238710,36.498898,0.508124,1.055974,1.324158,1.328669,0.489979,1.016454,1.272848,1.277103,0.018145,0.040853,0.054334,0.055103,0.588856,1.492747,2.120765,2.141576,0.433954,3.013173,6.563945,6.770772,0.508124,3.490748,7.675534,7.930801,0.489979,3.376237,7.436927,7.680199,0.018145,0.114511,0.238608,0.250602,4.230774e+06,2.768407e+07,6.311417e+07,6.546422e+07,4.106924e+06,2.699154e+07,6.165091e+07,6.392101e+07,123849.586140,692530.434671,1.463258e+06,1.543204e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.537968,0.433954,0.785074
no onboarding,243.977637,0.000167,0.886849,0.0,0.003505,0.633011,4.446095,9.498999,9.835113,2.308244,15.813084,35.836615,37.228972,0.500000,1.056075,1.324433,1.329272,0.481308,1.017523,1.272029,1.276535,0.018692,0.039887,0.055741,0.056075,0.575601,1.499166,2.142356,2.162550,0.426235,3.014352,6.642857,6.867156,0.50000

In [57]:
(engagement.groupby('marketing_holdout').mean().iloc[0,]/engagement.groupby('marketing_holdout').mean().iloc[1,]-1)*100

swid_holdout                                -51.749611
is_flagged                                   11.702936
is_entitled                                  -0.060381
is_pre_launch                                      NaN
is_pre_launch_nltt                            1.486609
total_login_days_l1                          -0.105340
total_login_days_l7                           0.102917
total_login_days_l28                          0.887629
total_login_days_itd                          0.935361
total_streams_l1                             -1.186059
total_streams_l7                              0.698722
total_streams_l28                             1.799762
total_streams_itd                             1.730909
num_streaming_profiles_l1                     0.476004
num_streaming_profiles_l7                     0.406390
num_streaming_profiles_l28                    0.281370
num_streaming_profiles_itd                    0.218603
num_general_streaming_profiles_l1             0.450535
num_genera

In [28]:
engagement.groupby('marketing_holdout').median()

,swid_holdout,total_streams_l1,total_streams_l7,is_entitled,num_streaming_devices_l1,num_streaming_devices_l7,num_streaming_profiles_l1,num_streaming_profiles_l7,active_binary_l1,active_binary_l7
marketing_holdout,,,,,,,,,,
all marketing,121,0,6,1,0,1,0,1,0,1
no marketing,250,0,5,1,0,1,0,1,0,1
no onboarding,244,0,6,1,0,1,0,1,0,1


In [ ]:
t1, p1 = stats.ttest_ind(engagement.loc[engagement['marketing_holdout']=='all marketing','is_entitled'],
                        engagement.loc[engagement.marketing_holdout=='no onboarding','is_entitled'])